In [ ]:
import csv
import nltk
import re
from datetime import datetime

nltk.download()

from nltk.tokenize import sent_tokenize , word_tokenize

In [45]:

def tokenizer(document):
    # define regular expression patterns
    punctuation_pattern = r'\w+|[^\w\s]' # matches all punctuations
    hashtag_pattern = r'#\w+' # matches hashtags
    user_pattern = r'@\w+' # matches user references
    smiley_pattern = r'[:;]-?[)(PD/\\]' # matches smileys
    date_pattern = r'(0?[1-9]|[12][0-9]|3[01])[/-](0?[1-9]|1[012])[/-]\d{2,4}|[a-zA-Z]+[\'’]?\d{2}|\d{2}[/-]\d{2}[/-]\d{2,4}' # matches dates
    time_pattern = r'\d{1,2}:\d{2}(?:(?:\s)?[AP]M)?(?:\s[A-Za-z]{2,4})?' # matches times

    # tokenize the document using regex
    tokens = nltk.regexp_tokenize(document, punctuation_pattern)

    # separate clitics into component words
    clitics = {
        "'m": "am", "'re": "are", "'ve": "have", "'ll": "will",
        "'s": "is", "'d": "would", "n't": "not"
    }
    new_tokens = []
    for token in tokens:
        if token in clitics:
            new_tokens.extend([clitics[token]])
        elif token.endswith("'s"):
            new_tokens.extend([token[:-2], "'s"])
        elif token.endswith("'"):
            new_tokens.extend([token[:-1], "'"])
        else:
            new_tokens.append(token)

    # separate possessive apostrophes
    new_tokens = [re.sub(r"(\w+)'(\w+)", r"\1 '\2", token) for token in new_tokens]

    # separate hashtags and user references
    new_tokens = [token.lower() for token in new_tokens]
    new_tokens = [re.sub(hashtag_pattern, r'\g<0> ', token) for token in new_tokens]
    new_tokens = [re.sub(user_pattern, r'\g<0> ', token) for token in new_tokens]

    # separate smileys
    new_tokens = [re.sub(smiley_pattern, r' \g<0> ', token) for token in new_tokens]

    # convert dates into canonical format
    new_tokens = [re.sub(date_pattern, lambda x: 'CF:D:' + canonical_date(x.group()), token) for token in new_tokens]

    # convert times into canonical format
    new_tokens = [re.sub(time_pattern, lambda x: 'CF:T:' + canonical_time(x.group()), token) for token in new_tokens]

    # remove extra whitespaces
    new_tokens = [re.sub('\s+', ' ', token) for token in new_tokens]

    return new_tokens

with open('reviews.csv', 'r') as file:
    
    # Create a CSV reader object
    reader = csv.reader(file)
    
    # Iterate over rows
    i = 0
    # Some errors here need to fix!
    for document in reader:
        if i%2==0 and i<40:
            sents = tokenize(document[1])
            print(sents)
            print("\n")
        if i>100:
            break
        i = i + 1 

['ubisoft', 'could', 'learn', 'a', 'thing', 'or', 'two', 'from', 'vaas', '', "'", 's', 'insanity', 'speech', '.', 'just', 'pirate', 'the', 'game', 'at', 'this', 'point', '.', 'you', '', "'", 'll', 'get', 'a', 'way', 'better', 'experience', '.']


['you', 'play', 'as', 'a', 'white', 'ass', 'kid', 'from', 'cali', 'who', 'goes', 'on', 'a', 'bloody', 'rampage', 'to', 'kill', 'as', 'many', 'minorities', 'as', 'possible', 'on', 'an', 'island', '.', 'good', 'game', '.']


['far', 'cry', '3', '.', 'far', 'cry', '3', '!', 'ubisoft', 'must', 'be', 'proud', 'of', 'this', 'game', '.', 'far', 'cry', '3', 'takes', 'the', 'open', '-', 'world', 'action', 'onto', 'a', 'completely', 'new', 'level', '.', 'you', 'can', 'either', 'spend', 'your', 'time', 'completing', 'missions', ',', 'overtaking', 'enemy', 'outposts', 'or', 'hunt', 'animals', '.', 'hunting', 'animals', 'isn', '', "'", 't', 'there', 'just', 'so', 'the', 'jungle', 'feels', 'alive', ',', 'it', 'actually', 'has', 'a', 'huge', 'purpouse', '.',